In [1]:
using LinearAlgebra
using StaticArrays

Precompiling StaticArrays
  ✓ StaticArrays
  1 dependency successfully precompiled in 6 seconds. 3 already precompiled.


In [2]:
struct Vector_2D{T<:Real} <: FieldVector{2, T}
    x::T
    y::T
end

In [3]:
Base. +(a::Vector_2D{T}, b::Vector_2D{T}) where T = Vector_2D{T}(a.x + b.x, a.y + b.y)
Base. -(a::Vector_2D{T}, b::Vector_2D{T}) where T = Vector_2D{T}(a.x - b.x, a.y - b.y)
Base. *(c::T, a::Vector_2D{T}) where T = Vector_2D{T}(c * a.x, c * a.y)

In [4]:
LinearAlgebra.norm(a::Vector_2D{T}) where T = sqrt(a.x^2 + a.y^2)
LinearAlgebra.dot(a::Vector_2D{T}, b::Vector_2D{T}) where T = a.x * b.x + a.y * b.y

In [5]:
Base. cos(a::Vector_2D{T}, b::Vector_2D{T}) where T = dot(a,b)/norm(a)/norm(b)
Base.sin(a::Vector_2D{T}, b::Vector_2D{T}) where T = xdot(a, b)/norm(a)/norm(b)
Base.angle(a::Vector_2D{T}, b::Vector_2D{T}) where T = atan(sin(a, b), cos(a, b))
Base.sign(a::Vector_2D{T}, b::Vector_2D{T}) where T = sign(sin(a, b))

In [6]:
xdot(a::Vector_2D{T}, b::Vector_2D{T}) where T = a.x * b.y - a.y * b.x

xdot (generic function with 1 method)

In [7]:
struct Segment_2D{T<:Real}
    A::Vector_2D{T}
    B::Vector_2D{T}
end

In [8]:
function is_one(P::Vector_2D{T}, Q::Vector_2D{T}, s::Segment_2D{T}) where T
    l = s.B-s.A
    return sin(l, P-s.A)*sin(l,Q-s.A)>0
end

is_one (generic function with 1 method)

In [9]:
is_one_area(F::Function, P::Vector_2D{T}, Q::Vector_2D{T}) where T = (F(P...)*F(Q...)>0)

is_one_area (generic function with 1 method)

In [10]:
y = Segment_2D{Int}(Vector_2D{Int}(0,0), Vector_2D{Int}(1,1))

Segment_2D{Int64}([0, 0], [1, 1])

In [11]:
a = Vector_2D{Int}(0,1)

2-element Vector_2D{Int64} with indices SOneTo(2):
 0
 1

In [12]:
b = Vector_2D{Int}(0, 2)

2-element Vector_2D{Int64} with indices SOneTo(2):
 0
 2

In [13]:
is_one(a, b, y)

true

In [14]:
b = Vector_2D{Int}(1, 0)

2-element Vector_2D{Int64} with indices SOneTo(2):
 1
 0

In [15]:
is_one(a, b, y)

false

In [16]:
function convex(vertices::Vector{Vector_2D{T}}) where T
    n = length(vertices)
    if n < 3 return true end
    arr = [vertices[i+1] - vertices[i] for i in 1:n-1]
    xdot_arr = [xdot(arr[i], arr[i+1]) for i in 1:n-2]

    return all(i -> i >= 0, xdot_arr) || all(i -> i <= 0, xdot_arr)
end

convex (generic function with 1 method)

In [73]:
a = Vector_2D{Float64}(0,0)
b = Vector_2D{Float64}(1,0)
c = Vector_2D{Float64}(1,1)
d = Vector_2D{Float64}(0,1)
convex([a, b, c, d])

true

In [18]:
function inside(vector::Vector_2D{T}, vertices::Vector{Vector_2D{T}}) where T
    res = zero(T)
    for i in 1:length(vertices)-1
        res += angle(vertices[i] - vector, vertices[i+1] - vector)
    end

    res += angle(vertices[end] - vector, vertices[begin] - vector)

    return res > π
end

inside (generic function with 1 method)

In [19]:
inside(Vector_2D(0.5, 0.5), [a, b, c, d])

true

In [20]:
function rotate(p1::Vector_2D{T}, p2::Vector_2D{T}, p3::Vector_2D{T}) where T
    return ((p2[1] - p1[1]) * (p3[2] - p1[2]) - (p2[2] - p1[2]) * (p3[1] - p1[1]))
end

rotate (generic function with 1 method)

In [48]:
function Jarvis(points::Vector{Vector_2D{T}}) where T
    n = length(points)
    if n < 3 return points end
    P = [i for i in 1:n]

    for i in 1:n
        if points[P[i]].x < points[P[1]].x
            P[i], P[1] = P[1], P[i]
        end
    end
    
    H = [P[1]]
    deleteat!(P, 1)
    append!(P, H[1])

    while true
        right = 1
        for i in 2:length(P)
            if rotate(points[H[end]], points[P[right]], points[P[i]]) < 0
                right = i
            end
        end
        if P[right] == H[begin]
            break
        else
            append!(H, P[right])
            deleteat!(P, right)
        end
    end

    return [points[i] for i in H]
end

Jarvis (generic function with 1 method)

In [49]:
p1 = Vector_2D(0.2, 0.2)
p2 = Vector_2D(0.3, 0.3)
p3 = Vector_2D(0.2, 0.5)
p4 = Vector_2D(0.3, 0.1)
p5 = Vector_2D(0.1, 0.1)
p6 = Vector_2D(0.4, 0.2)
points = [p1, p2, p3, p4, p5, p6]
Jarvis(points)

4-element Vector{Vector_2D{Float64}}:
 [0.1, 0.1]
 [0.3, 0.1]
 [0.4, 0.2]
 [0.2, 0.5]

In [66]:
function graham(points::Vector{Vector_2D{T}}) where T
    n = length(points)
    P = [i for i in 1:n]

    for i in 1:n
        if points[P[i]].x < points[P[begin]].x
            P[i], P[1] = P[1], P[i]
        end
    end
    for i in 2:n
        j = i
        while j > 1 && (rotate(points[P[begin]], points[P[j-1]], points[P[j]]) < 0)
            P[j], P[j-1] = P[j-1], P[j]
            j -= 1
        end
    end
    S = [P[1], P[2]]
    for i in 2:n
        while rotate(points[S[end-1]], points[S[end]], points[P[i]]) < 0
            pop!(S)
        end
        append!(S, P[i])
    end
    return [points[i] for i in S]
end

graham (generic function with 1 method)

In [67]:
graham(points)

5-element Vector{Vector_2D{Float64}}:
 [0.1, 0.1]
 [0.3, 0.1]
 [0.3, 0.1]
 [0.4, 0.2]
 [0.2, 0.5]

In [85]:
function s_t(vertices::Vector{Vector_2D{T}}) where T
    n = length(vertices)
    area = 0

    for i in 1:n
        x1, y1 = vertices[i]
        x2, y2 = vertices[(i % n) + 1]
        area += (x1 + x2) * (y2 - y1)
    end
    return area/2
end

s_t (generic function with 1 method)

In [86]:
a = Vector_2D{Float64}(0,0)
b = Vector_2D{Float64}(1,0)
c = Vector_2D{Float64}(1,1)
d = Vector_2D{Float64}(0,1)

2-element Vector_2D{Float64} with indices SOneTo(2):
 0.0
 1.0

In [87]:
s_t([a, b, c, d])

1.0

In [88]:
function s_triangle(vertices::Vector{Vector_2D{T}}) where T
    n = length(vertices)
    area = 0

    for i in 1:n-1
        area += xdot(vertices[i], vertices[i+1])
    end

    area += xdot(vertices[n], vertices[begin])

    return area/2
end

s_triangle (generic function with 1 method)

In [89]:
s_triangle([a, b, c, d])

1.0